In [1]:
!pip install htrc-feature-reader
# !pip install htrc
# !pip install isbnlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 13.2 MB/s eta 0:00:00
  Created wheel for htrc-feature-reader: filename=htrc_feature_reader-2.0.7-py3-none-any.whl size=40406 sha256=dea9e7c2abb1d7a358f4a3bc3391c5c275bed90b430c921e48ea16fcae27b429
  Stored in directory: /root/.cache/pip/wheels/b3/f1/1f/5c294023ab216db36f2d0106839e71b85612d467226e0a01ee
Successfully built htrc-feature-reader


In [102]:
from lxml import etree
import os
from urllib.request import urlopen
import json
from htrc_features import *
from htrc import workset, metadata
import pandas as pd
from lxml import etree
from time import sleep
from tqdm import tqdm
from isbnlib import mask, to_isbn10, is_isbn10, is_isbn13
import re
import linecache
import time
import json

def get_link(link):
    r = requests.get(link)
    return(r)

def get_htid(link):
    r = get_link(link)
    data = r.json()
    if len(data["items"]) > 0:
      matched = True
      htids = [(value['htid']) for value in data["items"]]
      year = [(value['publishDates']) for value in data["records"].values()][0]
      title = [(value['titles']) for value in data["records"].values()][0]
    else:
      matched = False
      htids = []
      year = 0
      title = ""
    return(htids, year, title, matched)

def get_subject(link):
    r = get_link(link)
    data = r.json()
    result = [(value['marc-xml']) for value in data["records"].values()][0]
    result = result.replace('encoding="UTF-8"', "")
    root = etree.fromstring(result)
    subject_list = []
    for record in root:
      fields = ['600', '610', '611', '630', '650', '651', '655']
      subject_no = 0
      for field in fields:
        query = "".join(['{http://www.loc.gov/MARC21/slim}datafield[@tag="', field, '"]'])
        subjects = record.findall(query)
        for item in subjects:
          subfields = item.findall('{http://www.loc.gov/MARC21/slim}subfield')
          Ind2 = item.get("ind2")
          if Ind2 == "0":
            for s in subfields:
              if s.get('code') == 'a':
                  value = s.text
                  subject_list.append(value)
    return([subject_list, subject_no])


# Open file and read line by line
file1 = open('/content/classics_list.txt', 'r')

count = sum( 1 for line in file1 )
print(count)

1000


In [114]:
data_summary = []
htid_lists = []
subject_lists = []

for i in tqdm(range(1, count)):
    # Get next line
    url = linecache.getline('/content/classics_list.txt', i)
    json_file_name = url.split("/")[5].split(".")[0]

    # if line is empty
    # end of file is reached
    if not url:
        break

    # store the response of URL
    response = urlopen(url)

    # storing the JSON response
    # from url in data
    data_json = json.loads(response.read())
    if data_json['availability']['Open Library']:
        open_lib_url = data_json['availability']['Open Library']
        print(open_lib_url)

        #https://openlibrary.org/search.json
        new_open_lib_url = open_lib_url.replace('search?', 'search.json?')

        r = requests.get(new_open_lib_url)
        ol_data = r.json()
        ol_filename = "".join(['./open_library_json/', json_file_name, '.json'])
        try:
          with open(ol_filename, 'w+') as file:
            json.dump(ol_data, file)
        except FileExistsError:
          print(f"The file '{file_path}' already exists.")


        # store the response of URL
        response = urlopen(new_open_lib_url)
        data_json = json.loads(response.read())

        try:
          rnumbers = data_json['docs'][0]['id_hathi_trust']
        except:
          rnumbers = []
        try:
          isbns = data_json['docs'][0]['isbn']
        except:
          isbns = []

        rnumber_count = len(rnumbers)
        isbn_count = len(isbns)

        if rnumber_count > 0:
          print(rnumbers[0])
          link = "".join(["https://catalog.hathitrust.org/api/volumes/full/recordnumber/",
                          rnumbers[0], ".json"])
        elif isbn_count > 0:
          print(isbns[0])
          link = "".join(["https://catalog.hathitrust.org/api/volumes/full/isbn/",
                          isbns[0], ".json"])
        print(link)
        htids = get_htid(link)

        r = requests.get(link)
        data = r.json()
        ht_filename = "".join(['./hathitrust_json/', json_file_name, '.json'])
        try:
          with open(ht_filename, 'w+') as file:
            json.dump(data, file)
        except FileExistsError:
          print(f"The file '{file_path}' already exists.")

        data_summary.append({
            "osurl":url,
            "ht_link":link,
            "ht_matched":htids[3],
            "htid_count":len(htids[0]),
            "year":htids[1],
            'title':htids[2],
            "subject_no":subjects[1]
        })

        if htids[3] == True:

          subjects = get_subject(link)

          for item in htids[0]:
              htid_lists.append({
                  "osurl":url,
                  "ht_link":link,
                  "htid":item
              })
          for item in subjects[0]:
            subject_lists.append({
                "osurl":url,
                "subject":item
            })


        time.sleep(2)


print(data_summary)
print(htid_lists)
print(subject_lists)

file1.close()




  0%|          | 0/999 [00:00<?, ?it/s]

https://openlibrary.org/search?title=Metamorphoses&author=Ovid
009709406
https://catalog.hathitrust.org/api/volumes/full/recordnumber/009709406.json


  0%|          | 1/999 [00:07<1:59:57,  7.21s/it]

https://openlibrary.org/search?title=Basics+of+Biblical+Greek+Grammar&author=William+D.+Mounce
9780310232117
https://catalog.hathitrust.org/api/volumes/full/isbn/9780310232117.json


  0%|          | 2/999 [00:10<1:40:47,  6.07s/it]

...

https://openlibrary.org/search?title=Pictorial+Dictionary+of+Ancient+Athens&author=John+Travlos
087817267X
https://catalog.hathitrust.org/api/volumes/full/isbn/087817267X.json


100%|█████████▉| 998/999 [1:07:47<00:03,  3.86s/it]

https://openlibrary.org/search?title=Sexual+Morality+in+Ancient+Rome&author=Rebecca+Langlands
9780511482823
https://catalog.hathitrust.org/api/volumes/full/isbn/9780511482823.json


100%|██████████| 999/999 [1:07:51<00:00,  4.08s/it]

[{'osurl': 'https://explorer-api.opensyllabus.org/v1/works/7507602833485.json\n', 'ht_link': 'https://catalog.hathitrust.org/api/volumes/full/recordnumber/009709406.json', 'ht_matched': True, 'htid_count': 1, 'year': ['1613'], 'title': ['The scourge of Venus, or the wanton lady : with the rare birth of Adonis'], 'subject_no': 0}, {'osurl': 'https://explorer-api.opensyllabus.org/v1/works/33268816675126.json\n', 'ht_link': 'https://catalog.hathitrust.org/api/volumes/full/isbn/9780310232117.json', 'ht_matched': True, 'htid_count': 1, 'year': ['1999'], 'title': ['Basics of biblical Greek : grammar'], 'subject_no': 0}, {'osurl': 'https://explorer-api.opensyllabus.org/v1/works/7507602833417.json\n', 'ht_link': 'https://catalog.hathitrust.org/api/volumes/full/isbn/9781788884167.json', 'ht_matched': False, 'htid_count': 0, 'year': 0, 'title': '', 'subject_no': 0}, {'osurl': 'https://explorer-api.opensyllabus.org/v1/works/7507602833416.json\n', 'ht_link': 'https://catalog.hathitrust.org/api/vol

In [115]:
data_summary_df = pd.DataFrame(data_summary)

print(len(data_summary))
print(sum(data_summary_df['ht_matched']))
print(len(htid_lists))
print(len(subject_lists))

999
306
593
733


In [116]:
data_summary_df.head()

,osurl,ht_link,ht_matched,htid_count,year,title,subject_no
0,https://explorer-api.opensyllabus.org/v1/works...,https://catalog.hathitrust.org/api/volumes/ful...,True,1,[1613],"[The scourge of Venus, or the wanton lady : wi...",0
1,https://explorer-api.opensyllabus.org/v1/works...,https://catalog.hathitrust.org/api/volumes/ful...,True,1,[1999],[Basics of biblical Greek : grammar],0
2,https://explorer-api.opensyllabus.org/v1/works...,https://catalog.hathitrust.org/api/volumes/ful...,False,0,0,,0
3,https://explorer-api.opensyllabus.org/v1/works...,https://catalog.hathitrust.org/api/volumes/ful...,False,0,0,,0
4,https://explorer-api.opensyllabus.org/v1/works...,https://catalog.hathitrust.org/api/volumes/ful...,False,0,0,,0


In [118]:
htid_df = pd.DataFrame(htid_lists)
htid_df.head()

,osurl,ht_link,htid
0,https://explorer-api.opensyllabus.org/v1/works...,https://catalog.hathitrust.org/api/volumes/ful...,hvd.32044090286501
1,https://explorer-api.opensyllabus.org/v1/works...,https://catalog.hathitrust.org/api/volumes/ful...,mdp.39015055612231
2,https://explorer-api.opensyllabus.org/v1/works...,https://catalog.hathitrust.org/api/volumes/ful...,ucbk.ark:/28722/h2028pg1w
3,https://explorer-api.opensyllabus.org/v1/works...,https://catalog.hathitrust.org/api/volumes/ful...,mdp.39015056249272
4,https://explorer-api.opensyllabus.org/v1/works...,https://catalog.hathitrust.org/api/volumes/ful...,mdp.39015058708507


In [119]:
subject_df = pd.DataFrame(subject_lists)
subject_df.head()

,osurl,subject
0,https://explorer-api.opensyllabus.org/v1/works...,Bible.
1,https://explorer-api.opensyllabus.org/v1/works...,"Greek language, Biblical"
2,https://explorer-api.opensyllabus.org/v1/works...,Aeneas (Legendary character)
3,https://explorer-api.opensyllabus.org/v1/works...,Bible.
4,https://explorer-api.opensyllabus.org/v1/works...,"Greek language, Biblical"


In [120]:
data_summary_df.to_csv("data_summary.csv")
htid_df.to_csv("htid_list.csv")
subject_df.to_csv("subject_list.csv")